# Degradation and Acceleration Factors
This tool will provide a simple method for estimating degradation and for calculating acceleration factors. It interfaces with the degradation database to simplify acquisition of the degradation parameters.

**Requirements**:
- compatible weather file (e.g., PSM3, TMY3, EPW...)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
    - & etc.
- Activation energies for test material [kJ/mol]
- Other degradation parameters

**Objectives**:
1. Read in the weather data
2. Gather basic degradation modeling data for a material of interest
3. Calculate absolute degradation rate
4. Run Monte Carlo simulation at a single site
5. Generate chamber or field data for environmental comparison
6. Calculate degradation acceleration factor of field location to chamber (or another location)
7. Run Monte Carlo simulation of the acceleration factor for a single site
8. Select a region of interest and specific data points
9. Produce a map of acceleration factors for a region

In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.4.2

In [2]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
import numpy as np
import json
import pvlib

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)
print(DATA_DIR)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.4
pvdeg version  0.1.dev1152+gfbac6aa1f.d20250917
C:\Users\mprillim\sam_dev\PVDegradationTools\pvdeg\data


## 1. Read In the Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [4]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
weather_df, meta = pvdeg.weather.read(weather_file,'csv')
print(weather_file)
print(meta)
print(weather_df.head())

C:\Users\mprillim\sam_dev\PVDegradationTools\pvdeg\data\psm3_demo.csv
{'Source': 'NSRDB', 'Location ID': 145809.0, 'City': 'West Pleasant View', 'State': 'Colorado', 'Country': 'United States', 'Clearsky DHI Units': 'w/m2', 'Clearsky DNI Units': 'w/m2', 'Clearsky GHI Units': 'w/m2', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Solar Zenith Angle Units': 'Degree', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Relative Humidity Units': '%', 'Precipitable Water Units': 'cm', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Cloud Type -15': 'N/A', 'Cloud Type 0': 'Clear', 'Cloud Type 1': 'Probably Clear', 'Cloud Type 2': 'Fog', 'Cloud Type 3': 'Water', 'Cloud Type 4': 'Super-Cooled Water', 'Cloud Type 5': 'Mixed', 'Cloud Type 6': 'Opaque Ice', 'Cloud Type 7': 'Cirrus', 'Cloud Type 8': 'Overlapping', 'Cloud Type 9': 'Overshooting', 'Cloud Type 10': 'Unknown', 'Cloud Type 11': 'Dust', 'Cloud Type 12': 'Smoke', 'Fill Flag 0': 'N/

In [5]:
# This routine will get a meteorological dataset from anywhere in the world where it is available 
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
#weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
weather_id = (33.4152, -111.8315) #Mesa, Arizona
#weather_id = (0,0) # Somewhere else you are interested in.
weather_df, meta = pvdeg.weather.get_anywhere(id=weather_id)
print(meta)
display(weather_df)

{'Source': 'NSRDB', 'Location ID': '77855', 'City': 'Mesa', 'State': 'Arizona', 'Country': 'United States', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Surface Albedo Units': 'N/A', 'Version': '3.2.0', 'latitude': 33.41, 'longitude': -111.82, 'altitude': 381, 'tz': -7, 'wind_height': 2, 'house_number': '501', 'retail': 'Old West Plaza', 'ISO3166-2-lvl4': 'US-AZ', 'Street': 'East 2nd Avenue', 'County': 'Maricopa County', 'Zipcode': '85204', 'Country Code': 'US'}


,Year,Month,Day,Hour,Minute,dew_point,dhi,dni,ghi,albedo,pressure,temp_air,wind_direction,wind_speed,relative_humidity
2012-01-01 00:30:00-07:00,2012,1,1,0,30,-2.0,0.0,0.0,0.0,0.16,970.0,7.0,47.0,4.4,52.747050
2012-01-01 01:30:00-07:00,2012,1,1,1,30,-2.0,0.0,0.0,0.0,0.16,970.0,7.0,46.0,4.2,52.747050
2012-01-01 02:30:00-07:00,2012,1,1,2,30,-3.0,0.0,0.0,0.0,0.16,970.0,6.0,45.0,4.1,52.471167
2012-01-01 03:30:00-07:00,2012,1,1,3,30,-3.0,0.0,0.0,0.0,0.16,970.0,6.0,48.0,4.0,52.471167
2012-01-01 04:30:00-07:00,2012,1,1,4,30,-4.0,0.0,0.0,0.0,0.16,970.0,6.0,50.0,3.8,48.696633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998-12-31 19:30:00-07:00,1998,12,31,19,30,1.0,0.0,0.0,0.0,0.16,970.0,13.0,34.0,1.5,43.926257
1998-12-31 20:30:00-07:00,1998,12,31,20,30,0.0,0.0,0.0,0.0,0.16,970.0,11.0,41.0,2.2,46.618757
1998-12-31 21:30:00-07:00,1998,12,31,21,30,0.0,0.0,0.0,0.0,0.16,970.0,10.0,45.0,3.3,49.831135
1998-12-31 22:30:00-07:00,1998,12,31,22,30,-1.0,0.0,0.0,0.0,0.16,970.0,9.0,46.0,4.1,49.550248


#### POA Irradiance
Next we need to calculate the stress parameters including temperature and humidity. We start with POA irradiance.
Irradiance_kwarg governs the array orientation for doing the POA calculations. 
It is defaulted to a north-south single axis tracking. A fixed tilt set of parameters is included but is blocked out. 
Look in spectral.py and/or PVLib here for 1-axis kwargs, 
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.tracking.singleaxis.html#pvlib.tracking.singleaxis  
and for fixed tilt,  
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.irradiance.gti_dirint.html?highlight=poa .  

In [6]:
#irradiance_kwarg ={
    #"tilt": None,
    #"azimuth": None,
    #"module_mount": 'fixed'}
irradiance_kwarg ={
    "axis_tilt": None,
    "axis_azimuth": None,
    "module_mount": '1_axis'}
poa_df = pvdeg.spectral.poa_irradiance(weather_df=weather_df, meta=meta, **irradiance_kwarg)
#display(poa_df)

The array axis_azimuth was not provided, therefore an azimuth of 180.0 was used.
The array axis_tilt was not provided, therefore an axis tilt of 0° was used.



#### Get Spectrally Resolved Irradiance Data
This first set of commands will calculate spectrally resolved irradiance data. This may or may not be needed for a given degradation model and can be skipped here.

In [ ]:
# this whole block needs to be replaced with call to calculate spectrally resolved irradiance.

from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
#display(data)
print(INPUT_SPECTRA)

#Test function 
#cusotm_albedo['Summer']
#custom_albedo['Winter']
# custom_albedo['Snow']
#defaults - Grass, Dry Grass, Snow
#Flexible to add complexity later
custom_albedo_summer = 'A006'
custom_albedo_winter = { #required: startDate, wavelength (if len(albedo) > 1), albedo, isSnow defaults to False
    "data_entry_person": "Michael Kempe",
    "date_entered": "7/28/2025",
    "DOI": "10.3390/ijerph15071507",
    "source_title": "Ultraviolet Radiation Albedo and Reflectance in Review: The Influence to Ultraviolet Exposure in Occupational Settings",
    "authors": "Joanna Turner, Alfio V. Parisi",
    "reference": "Turner J, Parisi AV. Ultraviolet Radiation Albedo and Reflectance in Review: The Influence to Ultraviolet Exposure in Occupational Settings. Int J Environ Res Public Health. 2018 Jul 17;15(7):1507.",
    "keywords": "snow, ground",
    "months": "1,2,3,10,11,12",
    "startDate": "January 1", #Day of Year? 0-365 
    "HourOfYear": "1", #Hour of Year? 1-8760
    "isSnow": "False",
    "comments": "Data is emperically extrapolated from 280 nm to 297 nm. Data extracted from Turner et al. Figure 1 as a reference to Doda & Green Snow-Ground. Doda D., Green A. Surface Reflectance Measurements in the UV from an Airborne Platform. Part 1. Appl. Opt. 1980;19:2140-2145. doi: 10.1364/AO.19.002140. Doda D., Green A. Surface Reflectance Measurements in the Ultraviolet from an Airborne Platform. Part 2. Appl. Opt. 1981;20:636-642. doi: 10.1364/AO.20.000636.",
    "wavelength": "280, 297.32034, 300.02435, 301.8514, 305.79782, 310.10962, 313.21558, 317.4543, 322.86237, 329.9513, 331.19366, 339.89038, 343.06943, 350.34103, 360.02435, 369.96347, 380.3776, 386.77222, 390.0609, 400.14615",
    "albedo": "20, 29.515152, 28.30303, 29.454546, 28.90909, 34.696968, 36.757576, 39.363636, 39.21212, 38.60606, 41.272728, 40.909092, 42.242424, 42.21212, 40.575756, 43.21212, 43.090908, 43.454544, 43.60606, 39.757576"
}
#Startdate, albedo, wavelength -> then next one + boolean logic for snow ()
custom_albedo_snow = {}
# custom_albedo_snow 
spectra_folder = 'spectra' #If you have already pulled the spectra from SMARTS, pass the folder path to avoid going through the donwload process again. 
wavelengths = np.arange(280, 400, 25)  # Example wavelengths from 280 nm to 400 nm in steps of 25 nm
data = pvdeg.spectral.spectrally_resolved_irradiance(weather_df=weather_df, meta=meta, wavelengths=wavelengths, frontResultsOnly=None,
                                                     spectra_folder=spectra_folder, custom_albedo_summer=custom_albedo_summer, custom_albedo_winter=custom_albedo_winter, **irradiance_kwarg)
#return front, back, or both (True, False, None)
#bool frontResultsonly = True for front only
#separate columns for front and back irradiance: spectra_front: etc. , spectra_back: etc. (see spectra_pytest.csv)
#Check albedo boolean snow, winter non-snow, summer non-snow



C:\Users\mprillim\sam_dev\PVDegradationTools\tests\data\spectra_pytest.csv
path = c:\Users\mprillim\sam_dev\PVDegradationTools\tutorials_and_tools\tutorials_and_tools
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File
<class 'bifacial_radiance.main.MetObj'>.metadata:
{'latitude': 33.41, 'longitude': -111.82, 'elevation': 381, 'timezone': -7, 'city': '-', 'label': 'center'}
<class 'bifacial_radiance.main.MetObj'>.tmydata:
 <class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4314 entries, 2021-01-01 08:00:00-07:00 to 2021-12-31 16:00:00-07:00
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ghi         4314 non-null   float64
 1   dhi         431

Generating Spectral TMYs: 100%|███████████████████████████████████████| 5/5 [00:16<00:00,  3.36s/it]


                             GHI    DHI    DNI   Alb  DryBulb  Wspd
2021-01-01 08:00:00-07:00  146.0   39.0  643.0  0.16     12.0   4.0
2021-01-01 09:00:00-07:00  329.0   56.0  836.0  0.16     16.0   3.9
2021-01-01 10:00:00-07:00  479.0   65.0  920.0  0.16     20.0   4.0
2021-01-01 11:00:00-07:00  569.0   81.0  927.0  0.16     23.0   4.2
2021-01-01 12:00:00-07:00  602.0   83.0  936.0  0.16     25.0   4.1
...                          ...    ...    ...   ...      ...   ...
2021-12-31 12:00:00-07:00  474.0  199.0  499.0  0.16     19.0   2.6
2021-12-31 13:00:00-07:00  393.0  227.0  315.0  0.16     20.0   2.5
2021-12-31 14:00:00-07:00  468.0   65.0  896.0  0.16     21.0   2.2
2021-12-31 15:00:00-07:00  324.0   55.0  816.0  0.16     20.0   1.4
2021-12-31 16:00:00-07:00  145.0   38.0  627.0  0.16     19.0   0.8

[4314 rows x 6 columns]
The array axis_azimuth was not provided, therefore an azimuth of 180.0 was used.
The array axis_tilt was not provided, therefore an axis tilt of 0° was used.
 

100%|██████████| 4314/4314 [00:18<00:00, 230.27it/s]


Finished
Composite Data 1:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     2021-01-01 08:00:00-07:00  643.0   39.0    0.62     7.5  146.186638   
1     2021-01-01 09:00:00-07:00  836.0   56.0    0.62     8.5  328.909864   
2     2021-01-01 10:00:00-07:00  920.0   65.0    0.62     9.5  478.405036   
3     2021-01-01 11:00:00-07:00  927.0   81.0    0.62    10.5  569.570049   
4     2021-01-01 12:00:00-07:00  936.0   83.0    0.62    11.5  601.757142   
...                         ...    ...    ...     ...     ...         ...   
4309  2021-12-31 12:00:00-07:00  499.0  199.0    0.62    11.5  474.799224   
4310  2021-12-31 13:00:00-07:00  315.0  227.0    0.62    12.5  393.010834   
4311  2021-12-31 14:00:00-07:00  896.0   65.0    0.62    13.5  468.884006   
4312  2021-12-31 15:00:00-07:00  816.0   55.0    0.62    14.5  323.571934   
4313  2021-12-31 16:00:00-07:00  627.0   38.0    0.62    15.5  144.846977   

            inc        zen         azm  pvFront

...

Finished
Composite Data 4:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     2021-01-01 08:00:00-07:00  643.0   39.0    0.62     7.5  146.186638   
1     2021-01-01 09:00:00-07:00  836.0   56.0    0.62     8.5  328.909864   
2     2021-01-01 10:00:00-07:00  920.0   65.0    0.62     9.5  478.405036   
3     2021-01-01 11:00:00-07:00  927.0   81.0    0.62    10.5  569.570049   
4     2021-01-01 12:00:00-07:00  936.0   83.0    0.62    11.5  601.757142   
...                         ...    ...    ...     ...     ...         ...   
4309  2021-12-31 12:00:00-07:00  499.0  199.0    0.62    11.5  474.799224   
4310  2021-12-31 13:00:00-07:00  315.0  227.0    0.62    12.5  393.010834   
4311  2021-12-31 14:00:00-07:00  896.0   65.0    0.62    13.5  468.884006   
4312  2021-12-31 15:00:00-07:00  816.0   55.0    0.62    14.5  323.571934   
4313  2021-12-31 16:00:00-07:00  627.0   38.0    0.62    15.5  144.846977   

            inc        zen         azm  pvFront

100%|██████████| 4314/4314 [00:25<00:00, 170.37it/s]


Finished
Composite Data 5:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     2021-01-01 08:00:00-07:00  643.0   39.0    0.62     7.5  146.186638   
1     2021-01-01 09:00:00-07:00  836.0   56.0    0.62     8.5  328.909864   
2     2021-01-01 10:00:00-07:00  920.0   65.0    0.62     9.5  478.405036   
3     2021-01-01 11:00:00-07:00  927.0   81.0    0.62    10.5  569.570049   
4     2021-01-01 12:00:00-07:00  936.0   83.0    0.62    11.5  601.757142   
...                         ...    ...    ...     ...     ...         ...   
4309  2021-12-31 12:00:00-07:00  499.0  199.0    0.62    11.5  474.799224   
4310  2021-12-31 13:00:00-07:00  315.0  227.0    0.62    12.5  393.010834   
4311  2021-12-31 14:00:00-07:00  896.0   65.0    0.62    13.5  468.884006   
4312  2021-12-31 15:00:00-07:00  816.0   55.0    0.62    14.5  323.571934   
4313  2021-12-31 16:00:00-07:00  627.0   38.0    0.62    15.5  144.846977   

            inc        zen         azm  pvFront

c:\users\mprillim\sam_dev\bifacialvf\bifacialvf\bifacialvf.py:896: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  composite_data['G_' + str(header) + '_DHI_' + str(x)] = composite_data['Avg_Row' + str(fb) + 'GTI_DHIDirect'] / Sum_DHI.mask(Sum_DHI == 0) * weather_df['DHI_0' + str(x)]
c:\users\mprillim\sam_dev\bifacialvf\bifacialvf\bifacialvf.py:897: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  composite_data['G_' + str(header) + '_DNI_reflected_' + str(x)] = composite_data['Avg_Row' + str(fb) + 'GTI_DNIReflected'] / Sum_DNI_ALB.m

UnboundLocalError: cannot access local variable 'wavelength_str' where it is not associated with a value

#### Cell or Module Surface Temperature
The following will calculate the cell and module surface temperature using the King model as a default. Other models can be used as described at,  
https://pvlib-python.readthedocs.io/en/stable/reference/pv_modeling/temperature.html. The difference is less than one °C for ground mounted systems
but can be as high as 3 °C for a high temperature building integrated system.

Here the temperatures are added to the dataframe and the module temperature is selected as the default 'temperatue' for the degradation calculations. If it is a cell degradation that is being investigated, temp_cell should be used.

In [ ]:
temp_cell = pvdeg.temperature.cell(weather_df=weather_df, meta=meta, poa=poa_df)
temp_module = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_df)

weather_df['temp_cell'] = temp_cell
weather_df['temp_module'] = temp_module
weather_df['temperature'] = weather_df['temp_module']

#### Humidity
Depending on the component for which the calculation is being run on, the desired humidity may be the atmospheric humidity, the module surface humidity, the humidity in front of a cell with a permeable backsheet, the humidity in the backsheet, the humidity in the back encapsulant or another custom humidity location such as a diffusion limited location. The folowing are options for doing all of these calculations. Here all the different humidities are put in the weather_df dataframe, but to select one to be specifically used it should be named 'RH' for most degradation functions (check the documentation of a specific degradation calculation if in doubt). Here the surface humidity is selected as a default.

In [ ]:
RH_surface = pvdeg.humidity.surface_outside(weather_df['relative_humidity'], weather_df['temp_air'],temp_module)
RH_front_encapsulant = pvdeg.humidity.front_encap(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,encapsulant='W001')
#display(weather_df)
#display(RH_surface)
#display(temp_module)


Ce_back_encapsulant = pvdeg.humidity.Ce(
    temp_module = temp_module, 
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001')

display(type(Ce_back_encapsulant))
print(Ce_back_encapsulant)
display(Ce_back_encapsulant)
display(RH_front_encapsulant)
#RH_back_encapsulant = pvdeg.humidity.Ce(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,backsheet='W017',encapsulant='W001',output='rh')
#RH_backsheet = (RH_surface + RH_back_encapsulant) / 2

weather_df['RH_surface'] = RH_surface
weather_df['RH_front_encapsulant'] = RH_front_encapsulant
weather_df['Ce_back_encapsulant'] = Ce_back_encapsulant   
#weather_df['RH_back_encapsulant'] = RH_back_encapsulant
#weather_df['RH_backsheet'] = RH_backsheet

weather_df['RH'] = weather_df['RH_surface']

display(weather_df)



In [ ]:
display(Ce_back_encapsulant.type())


Each of the necessary arrays of data can be individually sent to a function for calculation in the function call, or they can be combined into a single dataframe. The degradation functions are set up to first check for a specific data set in the function call but if not found it looks for specific data or a suitable substitute in the weather dataframe.

Either the cell temperature or the module temperature can be used. 
If the model needs wavelength resolved spectra, that should be added as 'irradiance'. But for just intensity, the 'poa_global'

In [ ]:
#weather_df['temperature'] = temp_cell
weather_df['temperature'] = temp_module
weather_df['poa_global'] = poa_df['poa_global']


## 2. Gather Basic Degradation Modeling Data for a Material of Interest

First we need to gather in the parameters for the degradation process of interest. This includes things such as the activiation energy and parameters defining the sensitivity to moisture, UV light, voltage, and other stressors.
For this tutorial we will need solar position, POA, PV cell and module temperature. Let's gernate those individually with their respective functions.
The blocked out text will produce a list of key fields from the database for each entry.

In [ ]:
#kwarg_variables = pvdeg.utilities._read_material(name=None, fname="DegradationDatabase", item=("Material", "Equation", "KeyWords", "EquationType"))
#print(json.dumps(kwarg_variables, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('},' + "\n", '},' +'\n' +'\n'))
pvdeg.utilities.display_json(pvdeg_file="DegradationDatabase", fp=DATA_DIR)

This next set of codes will take the data from the extracted portion of the Json library and create a list of variables from it. If more variables need to be modified or added, this is where it should be done.

In [ ]:
deg_data = pvdeg.utilities.read_material(fp=DATA_DIR, key="D036", pvdeg_file="DegradationDatabase")
print(json.dumps(deg_data, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('\n'+'}', '}'))

Here we pull out the relevant variables and equation code identifier needed for running the calculations.

In [ ]:
func = "pvdeg.degradation." + deg_data["EquationType"]
print(func)


## 3. Calculate Absolute Degradation Rate

To do this calculation, we must have degradation parameter data for a process that is complete with all the necessary variables. 

In [ ]:
from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
display(data)
print(INPUT_SPECTRA)



## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

As with most `pvdeg` functions, the following functions will always require two arguments (weather_df and meta)

In [ ]:
# chamber irradiance (W/m^2)
I_chamber = 1000
# chamber temperature (C)
temp_chamber = 60

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber,
                                              poa=poa_df,
                                              temp=temp_cell)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META,
                                                   poa=poa_df,
                                                   temp=temp_cell)

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Equation:
$$ AF = N * \frac{ I_{chamber}^x * RH_{chamber}^n * e^{\frac{- E_a}{k T_{chamber}}} }{ \Sigma (I_{POA}^x * RH_{outdoor}^n * e^{\frac{-E_a}{k T_outdoor}}) }$$

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

Equation:
$$ I_{WA} = [ \frac{ \Sigma (I_{outdoor}^x * RH_{outdoor}^n e^{\frac{-E_a}{k T_{outdood}}}) }{ N * RH_{WA}^n * e^{- \frac{E_a}{k T_eq}} } ]^{\frac{1}{x}} $$

In [ ]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                poa=poa_df,
                                                temp=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    poa=poa_df,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    temp=temp_cell,
                                                    Ea=Ea)

## 5. Quick Method (Degradation)

For quick calculations, you can omit POA and both module and cell temperature. The function will calculate these figures as needed using the available weather data with the default options for PV module configuration.

In [ ]:
# chamber settings
I_chamber= 1000
temp_chamber=60
rh_chamber=15

# activation energy
Ea = 40

vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber)

irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META)

In [ ]:
rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    Ea=Ea)

## 6. Solder Fatigue

Estimate the thermomechanical fatigue of flat plate photovoltaic module solder joints over the time range given using estimated cell temperature. Like other `pvdeg` funcitons, the minimal parameters are (weather_df, meta). Running the function with only these two inputs will use default PV module configurations ( open_rack_glass_polymer ) and the 'sapm' temperature model over the entire length of the weather data. 

In [ ]:
fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META)

If you wish to reduce the span of time or use a non-default temperature model, you may specify the parameters manually. Let's try an explicit example.
We want the solder fatigue estimated over the month of June for a roof mounted glass-front polymer-back module.

1. Lets create a datetime-index for the month of June.
2. Next, generate the cell temperature. Make sure to explicity restrict the weather data to our dt-index for June. Next, declare the PV module configuration.
3. Calculate the fatigue. Explicity specify the time_range (our dt-index for June from step 1) and the cell temperature as we caculated in step 2

In [ ]:
# select the month of June
time_range = WEATHER.index[WEATHER.index.month == 6]

# calculate cell temperature over our selected date-time range.
# specify the module configuration
temp_cell = pvdeg.temperature.cell(weather_df=WEATHER.loc[time_range], meta=META,
                                   temp_model='sapm',
                                   conf='insulated_back_glass_polymer')


fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META,
                                       time_range = time_range,
                                       temp_cell = temp_cell)